### SQL Homework
Use this notebook to answer the questions.
It can be in the same project as the previous homework.  
When you are ready, **upload** to your github repo, and send me the link (just zip a txt file with the repo's address, and upload it as the homework).  
If your repo is private, invite me: balazs.balogh@cubixedu.com.

#### Import the SparkSession, create it then load the taxi data (yellow_tripdata_2024-08.parquet)

In [1]:

from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructType,
    StructField,
    IntegerType,
    StringType,
    DateType,
    TimestampType
)

spark = (
    SparkSession
    .builder
    .appName('SQL')
    .master('local[*]')
    .getOrCreate()
)


In [2]:

taxi_data_2024_09 = (
    spark
    .read
    .format("parquet")
    .load('../cubix_de_pyspark/data/yellow_tripdata_2024-09.parquet')
)

taxi_data_2024_09.createOrReplaceTempView("taxi_2024_09")

#### 1. What is the total fare amount for all trips?  
Please round the answer to two decimal places.

In [ ]:
spark.sql("""
SELECT 
        format_number(SUM(fare_amount), 2) AS total_fare_amount
FROM 
        taxi_2024_09
""").show()

# ROUND() doesn't work?

+-----------------+
|total_fare_amount|
+-----------------+
|    72,674,557.08|
+-----------------+



#### 2. Show the maximum fare amount, minimum fare amount, and average fare amount for each payment type. Order by payment type.
Round where you need to two decimal places.

In [13]:
spark.sql("""
SELECT 
        payment_type,
        MAX(fare_amount) AS maximum_fare_amount,
        MIN(fare_amount) AS minimum_fare_amount,
        format_number(AVG(fare_amount), 2) AS average_fare_amount
FROM 
        taxi_2024_09
GROUP BY
        payment_type
ORDER BY payment_type
""").show()


+------------+-------------------+-------------------+-------------------+
|payment_type|maximum_fare_amount|minimum_fare_amount|average_fare_amount|
+------------+-------------------+-------------------+-------------------+
|           0|             652.45|             -88.31|              19.47|
|           1|              500.0|             -323.0|              20.93|
|           2|             1862.2|             -999.0|              19.03|
|           3|              599.0|             -599.0|               6.02|
|           4|              999.0|             -999.0|               1.40|
+------------+-------------------+-------------------+-------------------+



#### 3. For trips with a fare amount greater than 20, what is the total tip amount for each day (based on the tpep_pickup_datetime)?
Round the tip to two decimal places, and order the results from highest total tip amount.  
Hint: Check DATE() function, to convert tpep_pickup_datetime to date, to get only the YYYY-MM-DD.

In [20]:
spark.sql("""
SELECT 
        DATE(tpep_pickup_datetime) AS date,
        CAST(SUM(tip_amount) AS numeric(10,2)) AS sum_tip_amount
FROM 
        taxi_2024_09
WHERE
        fare_amount > 20
GROUP BY
        date
ORDER BY
        sum_tip_amount DESC
""").show(10)


+----------+--------------+
|      date|sum_tip_amount|
+----------+--------------+
|2024-09-26|     288873.04|
|2024-09-19|     267403.31|
|2024-09-25|     259181.89|
|2024-09-12|     252716.63|
|2024-09-18|     246127.63|
|2024-09-22|     244425.39|
|2024-09-05|     243657.20|
|2024-09-20|     241722.96|
|2024-09-24|     241222.24|
|2024-09-23|     239764.14|
+----------+--------------+
only showing top 10 rows



#### 4. For each trip, show the fare amount along with a column that indicates if the trip was "expensive" (greater than 30) or "cheap" (less than or equal to 30).
Hint: Use CASE WHEN for deciding on expensive, or cheap.

#### 5. Find the first trip (based on tpep_pickup_datetime) for each VendorID and display the fare amount.
Hint: You can use CTE with ROW_NUMBER().

#### 7. Calculate the average trip distance for each VendorID, and assign a label of 'Above Average' or 'Below Average' for each trip based on the distance relative to the VendorID’s average trip distance.
Hint: CTE joined back to the main DataFrame.